<a href="https://colab.research.google.com/github/erichardson97/cell_crushers/blob/main/exploratory_data_analysis/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive, files
import pandas as pd
from typing import Union, Callable
import numpy as np
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.optimize import minimize
import pylab
import scipy.stats as stats
import os
import sklearn

drive.mount('/content/drive/')
os.chdir('/content/drive/MyDrive/CMIPB_Files')


In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform
distances = non_gex.applymap(lambda x:1 - abs(x))
distances = squareform(distances)
linkage_ = linkage(distances)
clusters = dict((p,v) for p,v in zip(non_gex.index, fcluster(linkage_, t = 0.2, criterion = 'distance')))
in_dataset_df['Cluster'] = in_dataset_df.index.map(clusters)

## Collinearity

# collinearity = ds.data_filtered.corr()
# type_feat = 'GEX'
# collinearity = ds.data_filtered.drop(['dataset'],axis=1).applymap(float).corr()

def subset(df, feature_list: list):
  return collinearity[feature_list].loc[feature_list]
sns.set(font_scale=1)
non_gex = subset(collinearity, list(in_dataset))
ax = sns.clustermap(non_gex, row_linkage = linkage_, col_linkage = linkage_, cbar_kws = {'label':'Correlation coefficient\n(Pearsons)'}, row_colors = in_dataset_df[['Differentiation', 'B cell activation', 'B-cell proliferation', 'Isotype switching', 'IgG', 'Activation', 'Plasma cell genes', 'Literature_features']].applymap(lambda x:(x, x, x)))

In [ ]:
from scipy.stats import t, linregress

def prediction_and_interval(x, y, alpha =.05):
    n = len(x)
    t_stat = t.ppf(1-alpha/2, df=n-2)
    r_object = linregress(x.values.astype(float), y.values.astype(float))
    regression_xrange = np.linspace(min(x), max(x)+.01, 100)
    y_pred = np.array([p*r_object.slope + r_object.intercept for p in x])
    ss = sum((x-np.mean(x))**2)
    se_regression = np.sqrt(
            np.square(y-y_pred).sum()/(n-2)
        )
    regression_line = pd.DataFrame()
    mean = np.mean(regression_xrange)
    df = pd.DataFrame()
    for i, value in enumerate(regression_xrange):
        se = se_regression * np.sqrt(
            1+1/n+np.square(value-mean)/ss
        )
        df.loc[i, 'x'] = value
        df.loc[i, 'y_pred'] = r_object.intercept+r_object.slope*value
        df.loc[i, 'lower'] = df.loc[i, 'y_pred']-t_stat*se
        df.loc[i, 'upper'] = df.loc[i, 'y_pred']+t_stat*se
    return df, r_object


In [ ]:
# @title Functions to look at between-batch correlation/ variance.



def spearman_ratio(data, feature_list, corr_func = spearmanr, target = 'Day14_IgG_Titre'):
  '''
  Calculates Spearman's correlation coefficients for feature list with
  target value, in 2020 and 2021 data. Returns df with
  ratio of max_corr to min_corr.
  '''
  values = {}
  for p in feature_list:
    data[p] = data[p].map(lambda x:float(x) if x != 'ND' else x)
  cohort1 = data[data['dataset']=='2020_dataset']
  cohort2 = data[data['dataset']=='2021_dataset']
  for p in feature_list:
    assert data[data[p]!='ND'][p].map(type).nunique() == 1
    assert data[data[p]!='ND'][p].map(type).unique()[0] == float
    cohort2_val = corr_func(cohort2[cohort2[p]!='ND'][p].values, cohort2[cohort2[p]!='ND'][target].values)
    cohort1_val = corr_func(cohort1[cohort1[p]!='ND'][p].values, cohort1[cohort1[p]!='ND'][target].values)
    total_val = corr_func(data[data[p]!='ND'][p], data[data[p]!='ND'][target])
    values[p] = {'2021_rho': cohort2_val[0],'2021_p':cohort2_val[1], '2020_rho':cohort1_val[0], '2020_p':cohort1_val[1],
                 'total_rho':total_val[0], 'total_p':total_val[1]}
  values = pd.DataFrame(values).fillna(0).T
  values['Ratio'] = values.apply(lambda x:calc_ratio(x['2020_rho'], x['2021_rho']), axis =1 )
  return values

def calc_ratio(val_2020, val_2021):
  if (val_2020 != 0) & (val_2021 != 0):
    return max([val_2020, val_2021]) / min([val_2020, val_2021])
  else:
    if val_2020 == 0:
      return val_2021
    else:
      return val_2020

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,5))
feature = 'GEX_AP2B1'
p = data[(data[feature]!='ND')&(data['Target']!='ND')]
p['Target'] = p['Target'].map(float)
p[feature] = p[feature].map(float)
sns.scatterplot(data = p, x = feature, y = 'Target', hue = 'dataset', ax=ax)
ax.set_ylabel('Z-normalized Day 14 IgG PT Titre')
ax.set_xlabel(f'Z-normalized {feature}')
regress2020, _ = prediction_and_interval(p[p['dataset']=='2020_dataset'][feature], p[p['dataset']=='2020_dataset']['Target'])
regress2021, _ = prediction_and_interval(p[p['dataset']=='2021_dataset'][feature], p[p['dataset']=='2021_dataset']['Target'])
sns.lineplot(data = regress2020, x = 'x', y = 'y_pred', c = 'C0')
ax.fill_between(regress2020['x'], regress2020['lower'], regress2020['upper'], color='C0', alpha =.1)
ax.fill_between(regress2021['x'], regress2021['lower'], regress2021['upper'], color='C1', alpha =.1)
sns.lineplot(data = regress2021, x = 'x', y = 'y_pred', c = 'C1')
ax.set_title('Opposing correlations')
rho, pval = spearmanr(p[p['dataset']=='2020_dataset'][feature], p[p['dataset']=='2020_dataset']['Target'])

ax.text(-2, 20, r'$\rho$'+f': {rho:.2f}, p: {pval:.2f}', bbox=dict(facecolor='white', edgecolor='C0'))

rho, pval = spearmanr(p[p['dataset']=='2021_dataset'][feature], p[p['dataset']=='2021_dataset']['Target'])
ax.text(-2, 15, r'$\rho$'+f': {rho:.2f}, p: {pval:.2f}', bbox=dict(facecolor='white', edgecolor='C1'))

In [ ]:
feature_list = list(data.columns)
data = data[data['Target']!='ND']
data['Target'] = data['Target'].map(float)
feature_list.remove('dataset')
feature_list.remove('Target')
values = spearman_ratio(data, feature_list, target = 'Target')
prefixes = set(['Titre','Cellfrequency','Cytokine','GEX'])
values['Type'] = values.index.map(lambda x:x.split('_')[0] if x.split('_')[0] in prefixes else 'NA' )
mean = values['Ratio'].mean()
std = np.std(values['Ratio'])
values['t_stat'] = (values['Ratio'].map(abs) - 1)/std
values[(values['t_stat']>2)|(values['t_stat']<-2)]
values.to_csv('RatioCorrCoefficients_Batch.csv')